In [1]:
import scanpy as sc
import pandas as pd
import anndata as ad
import anndata2ri
import os
from os.path import join
import scipy.sparse as sp
import numpy as np

from hlca_v2.ingestion_utils import get_gspread_df, ValidationWorkflow, AnnDataMerger

GSPREAD_JSON = "/home/icb/raphael.kfuri-rubens/data/hlca_v2/google_sheets_api/hlca-v2-8d5fea10d8f3.json"

Warning message:
package ‘methods’ was built under R version 4.3.3 
During startup - Warning messages:
1: package ‘datasets’ was built under R version 4.3.3 
2: package ‘utils’ was built under R version 4.3.3 
3: package ‘grDevices’ was built under R version 4.3.3 
4: package ‘graphics’ was built under R version 4.3.3 
5: package ‘stats’ was built under R version 4.3.3 


In [2]:
DATASET_ID = "Niklason_Kaminski_publ"
RDS_PATH = f"/home/icb/raphael.kfuri-rubens/data/hlca_v2/{DATASET_ID}/{DATASET_ID}.rds"
OUTPUT_PATH_PREREVISION = '/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_prerevision'
OUTPUT_PATH_POSTREVISION = '/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_postrevision'

In [3]:
# Name constants
AUTHOR_CELL_TYPE_L0 = 'author_cell_type_level_0'
AUTHOR_CELL_TYPE_L1 = 'author_cell_type_level_1'

CELL_TYPE_ONTOLOGY_ID_L0 = 'cell_type_ontology_term_id_level_0'
CELL_TYPE_ONTOLOGY_ID_L1 = 'cell_type_ontology_term_id_level_1'

CELL_TYPE_ONTOLOGY_LABEL_L0 = 'cell_type_ontology_term_label_level_0'
CELL_TYPE_ONTOLOGY_LABEL_L1 = 'cell_type_ontology_term_label_level_1'

AUTHOR_CELL_TYPE_DESCRIPTION_L0 = 'author_cell_type_description_level_0'
AUTHOR_CELL_TYPE_DESCRIPTION_L1 = 'author_cell_type_description_level_1'

MARKER_GENES_L0 = 'author_cell_type_markers_level_0'
MARKER_GENES_L1 = 'author_cell_type_markers_level_1'

# Finest grained annotation will be generic dataset cell type
AUTHOR_CELL_TYPE = 'author_cell_type'
CELL_TYPE_ONTOLOGY_ID = 'cell_type_ontology_term_id'
CELL_TYPE_ONTOLOGY_LABEL = 'cell_type_ontology_term_label'
MARKER_GENES = 'author_cell_type_markers'
AUTHOR_CELL_TYPE_DESCRIPTION = 'author_cell_type_description'

# Load data

In [4]:
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_4183780/2547887543.py:1: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [5]:
%%R -i RDS_PATH

suppressPackageStartupMessages(library(Seurat))
rds <- readRDS(file=RDS_PATH)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In addition: Warning messages:
1: package ‘tools’ was built under R version 4.3.3 
2: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/ictstr01/home/icb/raphael.kfuri-rubens/R/x86_64-redhat-linux-gnu-library/4.3’ contains no packages


In [6]:
%%R -o adata

adata <- as.SingleCellExperiment(rds)
adata

class: SingleCellExperiment 
dim: 46204 17867 
metadata(0):
assays(3): counts logcounts scaledata
rownames(46204): DPM1 SCYL3 ... ENSG00000277705 ENSG00000280139
rowData names(0):
colnames(17867): NDRI_1b__AAACCTGAGGATATAC NDRI_1b__AAAGATGGTCTGGAGA
  ... 388C_GTGCGGTCACCTATCC 92C_GAATGAATCTCTTATG
colData names(22): nGene nUMI ... Cell_Ontology_Class_ID ident
reducedDimNames(3): CCA CCA.ALIGNED TSNE
mainExpName: RNA
altExpNames(0):


/home/icb/raphael.kfuri-rubens/miniconda3/envs/hlcav2-env/lib/python3.10/site-packages/anndata2ri/_r2py.py:132: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm, layers=layers)


In [7]:
adata.obs

,nGene,nUMI,orig.ident,percent.mito,exp,species,class,library.ident,disease.ident,subject.ident,...,cell.cat.ident,name_initial,var.ratio.pca,hum.stash,nCount_RNA,nFeature_RNA,stash,Annotation_for_Publication,Cell_Ontology_Class_ID,ident
NDRI_1b__AAACCTGAGGATATAC,2607,8348.0,NDRI_1b,0.026007,Hum2,Human,Original CCA,None,None,Hum2,...,None,None,0.950244,Mo,8344.0,2603,Mo,Mo,CL:0000576,CL:0000576
NDRI_1b__AAAGATGGTCTGGAGA,1496,3905.0,NDRI_1b,0.050192,Hum2,Human,Original CCA,None,None,Hum2,...,None,None,1.057564,Mo,3905.0,1496,Mo,Mo,CL:0000576,CL:0000576
NDRI_1b__AAAGTAGGTGTGGTTT,2776,9400.0,NDRI_1b,0.044371,Hum2,Human,Original CCA,None,None,Hum2,...,None,None,1.067565,Mo,9398.0,2774,Mo,Mo,CL:0000576,CL:0000576
NDRI_1b__AAGGAGCAGTCTTGCA,1261,3436.0,NDRI_1b,0.036671,Hum2,Human,Original CCA,None,None,Hum2,...,None,None,1.033633,Mo,3436.0,1261,Mo,Mo,CL:0000576,CL:0000576
NDRI_1b__AAGTCTGCAATGAAAC,2507,8426.0,NDRI_1b,0.026110,Hum2,Human,Original CCA,None,None,Hum2,...,None,None,0.989638,Mo,8426.0,2507,Mo,Mo,CL:0000576,CL:0000576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222C_GGCTGGTAGCTGATAA,1211,2697.0,CellSoup,0.041898,None,None,Human Controls,222C,Control,222C,...,Myeloid,Mast,0.985601,Mast,2697.0,1211,Mast,Mast,CL:0000097,CL:0000097
388C_GCTGGGTCAGACACTT,1217,2522.0,CellSoup,0.055908,None,None,Human Controls,388C,Control,388C,...,Myeloid,Mast,0.985205,Mast,2522.0,1217,Mast,Mast,CL:0000097,CL:0000097
388C_GGACGTCAGCAGGTCA,922,1851.0,CellSoup,0.060508,None,None,Human Controls,388C,Control,388C,...,Myeloid,Mast,0.947162,Mast,1851.0,922,Mast,Mast,CL:0000097,CL:0000097
388C_GTGCGGTCACCTATCC,1074,2025.0,CellSoup,0.070123,None,None,Human Controls,388C,Control,388C,...,Myeloid,Mast,0.986631,Mast,2025.0,1074,Mast,Mast,CL:0000097,CL:0000097


In [8]:
obs = get_gspread_df(GSPREAD_JSON, DATASET_ID, "tier_1", "obs")
uns = get_gspread_df(GSPREAD_JSON, DATASET_ID, "tier_1", "uns")

In [9]:
val_workflow = ValidationWorkflow(
    input = uns,
    axis = 'uns'
)

validated_uns = val_workflow.init_workflow()
validated_uns

Validation workflow successful. Returning UNS dataframe.

,title,study_PI,batch_condition,default_embedding,unpublished,comments
0,All - Single Cell Connectomic Analysis of Adul...,"Laura,E,Niklason, Naftali,Kaminski","subject.ident,library.ident,exp",hum_GEN2@reductions$tsne,consented for release,HC1 and NDRI_1b were dissociated by Dr. Rared...


In [10]:
val_workflow = ValidationWorkflow(
    input = obs,
    axis = 'obs'
)

validated_obs = val_workflow.init_workflow()
validated_obs

Validation error(s) encountered.

{
│   'sample_source': "Must be one of {'surgical donor', 'postmortem donor', 'organ donor'}. Invalid entries found.",
│   'sample_collection_method': "Must be one of {'biopsy', 'brush', 'surgical resection'}. Invalid entries found."
}

Error containing field: sample_source
Error message: Must be one of {'surgical donor', 'postmortem donor', 'organ donor'}. Invalid entries found.
Value counts:
N/A                 12
surgical donor       1
postmortem donor     1
Name: sample_source, dtype: int64

Error containing field: sample_collection_method
Error message: Must be one of {'biopsy', 'brush', 'surgical resection'}. Invalid entries found.
Value counts:
N/A                   12
surgical resection     2
Name: sample_collection_method, dtype: int64

Returning error-containing OBS dataframe.

,sample_ID,donor_id,protocol_URL,institute,sample_collection_site,sample_collection_relative_time_point,library_ID,library_ID_repository,author_batch_notes,organism_ontology_term_id,...,sequenced_fragment,sequencing_platform,is_primary_data,reference_genome,gene_annotation_version,alignment_software,intron_inclusion,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,development_stage_ontology_term_id
0,Hum1,SA_donor_1,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_1_YSM,N/A,HC1,GSM3926545,Dissociated by MSBR and sequenced fresh (separ...,NCBITaxon:9606,...,3 prime tag,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000242
1,Hum2,SA_donor_2,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_1_YSM,N/A,NDRI_1b,GSM3926546,Dissociated by MSBR and sequenced fresh (separ...,NCBITaxon:9606,...,3 prime tag,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000243
2,001C,SA_donor_3,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,001C,GSM4050097,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000237
3,048C,SA_donor_4,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,048C,GSM4050099,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000238
4,098C,SA_donor_5,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,098C,GSM4050101,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000239
5,133C,SA_donor_6,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,133C,GSM4050103,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000238
6,1372C,SA_donor_7,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,1372C,GSM4050105,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000237
7,137C,SA_donor_8,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,137C,GSM4050107,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000240
8,160C,SA_donor_9,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,160C,GSM4050109,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000241
9,192C,SA_donor_10,https://www.science.org/doi/10.1126/sciadv.aaw...,Yale School of Medicine,Site_2_BWH,N/A,192C,GSM4050111,"Dissociated by members of Rosas lab, frozen in...",NCBITaxon:9606,...,3 prime tag,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000241


# Validate obs and uns from adata

In [11]:
adata.obs.columns

Index(['nGene', 'nUMI', 'orig.ident', 'percent.mito', 'exp', 'species',
       'class', 'library.ident', 'disease.ident', 'subject.ident',
       'cell.type.desc.ident', 'cell.type.ident', 'cell.cat.ident',
       'name_initial', 'var.ratio.pca', 'hum.stash', 'nCount_RNA',
       'nFeature_RNA', 'stash', 'Annotation_for_Publication',
       'Cell_Ontology_Class_ID', 'ident'],
      dtype='object')

In [12]:
obs['sample_ID'].value_counts(dropna=False)

Hum1     1
Hum2     1
001C     1
048C     1
098C     1
133C     1
1372C    1
137C     1
160C     1
192C     1
222C     1
244C     1
388C     1
454C     1
Name: sample_ID, dtype: int64

In [13]:
adata.obs['subject.ident'].value_counts(dropna=False)

Hum2     2997
098C     2849
222C     2825
Hum1     2193
133C     2074
1372C    1291
192C      823
048C      583
160C      574
388C      502
001C      466
454C      363
137C      211
244C      116
Name: subject.ident, dtype: int64

In [14]:
# check non-intersecting values of obs['library_ID'] and adata.obs['subject.ident']

sub_id = adata.obs['subject.ident'].value_counts(dropna=False).index.tolist()
lib_id = obs['sample_ID'].value_counts().index.tolist()

# show non-intersecting values 
set(sub_id).symmetric_difference(set(lib_id))

set()

In [15]:
merger = AnnDataMerger(
    adata = adata,
    uns_df = uns
)

adata = merger.add_uns_metadata()

adata

AnnData object with n_obs × n_vars = 17867 × 46204
    obs: 'nGene', 'nUMI', 'orig.ident', 'percent.mito', 'exp', 'species', 'class', 'library.ident', 'disease.ident', 'subject.ident', 'cell.type.desc.ident', 'cell.type.ident', 'cell.cat.ident', 'name_initial', 'var.ratio.pca', 'hum.stash', 'nCount_RNA', 'nFeature_RNA', 'stash', 'Annotation_for_Publication', 'Cell_Ontology_Class_ID', 'ident'
    uns: 'title', 'study_PI', 'batch_condition', 'default_embedding', 'unpublished', 'comments'
    obsm: 'CCA', 'CCA.ALIGNED', 'X_tsne'
    layers: 'logcounts', 'scaledata'

In [16]:
merger = AnnDataMerger(
    adata = adata,
    obs_df = obs
)

adata = merger.add_obs_metadata(
    adata_col = 'subject.ident',
    df_col = 'sample_ID',
    skip = None
)

adata.obs['sample_ID'] = adata.obs['subject.ident']

adata.obs

,nGene,nUMI,orig.ident,percent.mito,exp,species,class,library.ident,disease.ident,subject.ident,...,sequencing_platform,is_primary_data,reference_genome,gene_annotation_version,alignment_software,intron_inclusion,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,development_stage_ontology_term_id,sample_ID
NDRI_1b__AAACCTGAGGATATAC,2607,8348.0,NDRI_1b,0.026007,Hum2,Human,Original CCA,None,None,Hum2,...,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000243,Hum2
NDRI_1b__AAAGATGGTCTGGAGA,1496,3905.0,NDRI_1b,0.050192,Hum2,Human,Original CCA,None,None,Hum2,...,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000243,Hum2
NDRI_1b__AAAGTAGGTGTGGTTT,2776,9400.0,NDRI_1b,0.044371,Hum2,Human,Original CCA,None,None,Hum2,...,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000243,Hum2
NDRI_1b__AAGGAGCAGTCTTGCA,1261,3436.0,NDRI_1b,0.036671,Hum2,Human,Original CCA,None,None,Hum2,...,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000243,Hum2
NDRI_1b__AAGTCTGCAATGAAAC,2507,8426.0,NDRI_1b,0.026110,Hum2,Human,Original CCA,None,None,Hum2,...,EFO:0008563,true,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000243,Hum2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222C_GGCTGGTAGCTGATAA,1211,2697.0,CellSoup,0.041898,None,None,Human Controls,222C,Control,222C,...,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000241,222C
388C_GCTGGGTCAGACACTT,1217,2522.0,CellSoup,0.055908,None,None,Human Controls,388C,Control,388C,...,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000241,388C
388C_GGACGTCAGCAGGTCA,922,1851.0,CellSoup,0.060508,None,None,Human Controls,388C,Control,388C,...,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000241,388C
388C_GTGCGGTCACCTATCC,1074,2025.0,CellSoup,0.070123,None,None,Human Controls,388C,Control,388C,...,EFO:0008563,false,GRCh38,v91,zUMIs v0.4,no,PATO:0000461,unknown,HsapDv:0000241,388C


# Add author cell type markers to UNS

In [17]:
marker_list = pd.read_csv("/ictstr01/home/icb/raphael.kfuri-rubens/data/hlca_v2/Niklason_Kaminski_publ/curated.marker.list.2024-05-21.txt", sep=";").T
marker_list[AUTHOR_CELL_TYPE] = marker_list.index
marker_list.rename(columns={0:MARKER_GENES}, inplace=True)
marker_list.reset_index(drop=True, inplace=True)

marker_list

,author_cell_type_markers,author_cell_type
0,AGER,ATI
1,SFTPC,ATII
2,CD79A,B_cells
3,JCHAIN,B_Jchain
4,KRT5,Basal
5,CCDC153,Ciliated
6,CD86,DC
7,CA4,EC_cap
8,PROX1,EC_lymph
9,VWF,EC_vasc


# Check author cell type annotations and Cell Ontology IDs

In [18]:
adata.obs.columns

Index(['nGene', 'nUMI', 'orig.ident', 'percent.mito', 'exp', 'species',
       'class', 'library.ident', 'disease.ident', 'subject.ident',
       'cell.type.desc.ident', 'cell.type.ident', 'cell.cat.ident',
       'name_initial', 'var.ratio.pca', 'hum.stash', 'nCount_RNA',
       'nFeature_RNA', 'stash', 'Annotation_for_Publication',
       'Cell_Ontology_Class_ID', 'ident', 'donor_id', 'protocol_URL',
       'institute', 'sample_collection_site',
       'sample_collection_relative_time_point', 'library_ID',
       'library_ID_repository', 'author_batch_notes',
       'organism_ontology_term_id', 'manner_of_death', 'sample_source',
       'sex_ontology_term_id', 'sample_collection_method', 'tissue_type',
       'sampled_site_condition', 'tissue_ontology_term_id', 'tissue_free_text',
       'sample_preservation_method', 'suspension_type', 'cell_enrichment',
       'cell_viability_percentage', 'cell_number_loaded',
       'sample_collection_year', 'assay_ontology_term_id',
       'librar

In [19]:
adata.obs[AUTHOR_CELL_TYPE] = adata.obs['Annotation_for_Publication']
adata.obs[AUTHOR_CELL_TYPE].value_counts()

T_cells         2018
ATII            1797
Mo              1718
Mac_alv         1710
DC              1343
T_CD8+          1292
NK              1189
Mac_inter       1116
Ciliated         966
Fib_Col14a1+     724
EC_lymph         687
ATI              672
EC_vasc          632
B_cells          617
EC_cap           309
Mast             272
Secretory        215
Fib_Col13a1+     196
B_Jchain+        171
Basal            122
SMCs             101
Name: author_cell_type, dtype: int64

In [20]:
adata.obs[CELL_TYPE_ONTOLOGY_ID] = adata.obs['Cell_Ontology_Class_ID']
adata.obs[CELL_TYPE_ONTOLOGY_ID].value_counts()

CL:0000084    2018
CL:0002063    1797
CL:0000576    1718
CL:0000583    1710
CL:0000451    1343
CL:0000795    1292
CL:0000623    1189
CL:4033043    1116
CL:0000064     966
CL:4028006     724
CL:0002138     687
CL:0002062     672
CL:4028002     632
CL:0000236     617
CL:4028003     309
CL:0000097     272
CL:0000151     215
CL:4028004     196
CL:0000786     171
CL:0002329     122
CL:0000192     101
Name: cell_type_ontology_term_id, dtype: int64

In [21]:
cell_type_ontology_mapping = adata.obs.set_index(AUTHOR_CELL_TYPE)[CELL_TYPE_ONTOLOGY_ID].to_dict()

marker_list[CELL_TYPE_ONTOLOGY_ID] = marker_list[AUTHOR_CELL_TYPE].map(cell_type_ontology_mapping)

missing_ontologies = {
    'B_Jchain': 'CL:0000236',
    'Fib_Col13a1': 'CL:0000057',
    'Fib_Col14a1': 'CL:0000057',
    'T_CD8': 'CL:0000625'
}

marker_list[CELL_TYPE_ONTOLOGY_ID] = marker_list[AUTHOR_CELL_TYPE].map(missing_ontologies).fillna(marker_list[CELL_TYPE_ONTOLOGY_ID])

marker_list

,author_cell_type_markers,author_cell_type,cell_type_ontology_term_id
0,AGER,ATI,CL:0002062
1,SFTPC,ATII,CL:0002063
2,CD79A,B_cells,CL:0000236
3,JCHAIN,B_Jchain,CL:0000236
4,KRT5,Basal,CL:0002329
5,CCDC153,Ciliated,CL:0000064
6,CD86,DC,CL:0000451
7,CA4,EC_cap,CL:4028003
8,PROX1,EC_lymph,CL:0002138
9,VWF,EC_vasc,CL:4028002


In [22]:
adata.uns[MARKER_GENES] = marker_list

adata

AnnData object with n_obs × n_vars = 17867 × 46204
    obs: 'nGene', 'nUMI', 'orig.ident', 'percent.mito', 'exp', 'species', 'class', 'library.ident', 'disease.ident', 'subject.ident', 'cell.type.desc.ident', 'cell.type.ident', 'cell.cat.ident', 'name_initial', 'var.ratio.pca', 'hum.stash', 'nCount_RNA', 'nFeature_RNA', 'stash', 'Annotation_for_Publication', 'Cell_Ontology_Class_ID', 'ident', 'donor_id', 'protocol_URL', 'institute', 'sample_collection_site', 'sample_collection_relative_time_point', 'library_ID', 'library_ID_repository', 'author_batch_notes', 'organism_ontology_term_id', 'manner_of_death', 'sample_source', 'sex_ontology_term_id', 'sample_collection_method', 'tissue_type', 'sampled_site_condition', 'tissue_ontology_term_id', 'tissue_free_text', 'sample_preservation_method', 'suspension_type', 'cell_enrichment', 'cell_viability_percentage', 'cell_number_loaded', 'sample_collection_year', 'assay_ontology_term_id', 'library_preparation_batch', 'library_sequencing_run', 'seq

# Check whether ENSEMBL IDs in var

In [23]:
adata.var

""
DPM1
SCYL3
C1orf112
FGR
CFH
...
ENSG00000227029
RN7SL782P
ENSG00000274532
ENSG00000277705


In [25]:
import mygene

def map_gene_names_to_ensembl(mixed_list):
    mg = mygene.MyGeneInfo()
    gene_names = [item for item in mixed_list if not item.startswith("ENS")]
    gene_info = mg.querymany(gene_names, scopes='symbol', fields='ensembl.gene', species='human')

    gene_to_ensembl = {}
    for gene in gene_info:
        if 'notfound' not in gene and 'ensembl' in gene:
            ensembl_id = gene['ensembl']['gene'] if isinstance(gene['ensembl'], dict) else gene['ensembl'][0]['gene']
            gene_to_ensembl[gene['query']] = ensembl_id

    mapped_list = [gene_to_ensembl.get(item, item) for item in mixed_list]
    return mapped_list


adata.var['ensembl_id'] = map_gene_names_to_ensembl(adata.var.index.tolist())

adata.var

4689 input query terms found dup hits:	[('BORCS8-MEF2B', 2), ('PMS2P4', 2), ('PMS2P1', 2), ('CRYBB2P1', 2), ('LRP5L', 2), ('TPTEP1', 2), ('
1133 input query terms found no hit:	['C1orf112', 'FAM214B', 'KIAA0100', 'TMEM159', 'CXorf56', 'ARNTL2', 'SARS', 'ADSS', 'FAM214A', 'KIAA


,ensembl_id
DPM1,ENSG00000000419
SCYL3,ENSG00000000457
C1orf112,C1orf112
FGR,ENSG00000000938
CFH,ENSG00000000971
...,...
ENSG00000227029,ENSG00000227029
RN7SL782P,ENSG00000239708
ENSG00000274532,ENSG00000274532
ENSG00000277705,ENSG00000277705


In [26]:
# 1218 gene names without ensembl IDs
adata.var[~adata.var['ensembl_id'].str.startswith('ENSG')]

,ensembl_id
C1orf112,C1orf112
FAM214B,FAM214B
KIAA0100,KIAA0100
TMEM159,TMEM159
CXorf56,CXorf56
...,...
MIR4419B,MIR4419B
FAM197Y10,FAM197Y10
RNMTL1P1,RNMTL1P1
RNU6-785P,RNU6-785P


In [27]:
adata.var['ensembl_id'] = adata.var['ensembl_id'].apply(lambda x: x if str(x).startswith('ENSG') else np.nan)

adata.var

,ensembl_id
DPM1,ENSG00000000419
SCYL3,ENSG00000000457
C1orf112,NaN
FGR,ENSG00000000938
CFH,ENSG00000000971
...,...
ENSG00000227029,ENSG00000227029
RN7SL782P,ENSG00000239708
ENSG00000274532,ENSG00000274532
ENSG00000277705,ENSG00000277705


In [28]:
adata.var['gene_symbol'] = adata.var.index
adata.var.index = adata.var['ensembl_id']
adata.var.index.name = 'index'

adata.var

,ensembl_id,gene_symbol
index,,
ENSG00000000419,ENSG00000000419,DPM1
ENSG00000000457,ENSG00000000457,SCYL3
NaN,NaN,C1orf112
ENSG00000000938,ENSG00000000938,FGR
ENSG00000000971,ENSG00000000971,CFH
...,...,...
ENSG00000227029,ENSG00000227029,ENSG00000227029
ENSG00000239708,ENSG00000239708,RN7SL782P
ENSG00000274532,ENSG00000274532,ENSG00000274532


In [37]:
adata.var['gene_symbol'] = adata.var['gene_symbol'].astype(str)
adata.var['ensembl_id'] =adata.var['ensembl_id'].astype(str)

adata.var.index = adata.var.index.astype(str)
adata.var

,ensembl_id,gene_symbol
index,,
ENSG00000000419,ENSG00000000419,DPM1
ENSG00000000457,ENSG00000000457,SCYL3
nan,nan,C1orf112
ENSG00000000938,ENSG00000000938,FGR
ENSG00000000971,ENSG00000000971,CFH
...,...,...
ENSG00000227029,ENSG00000227029,ENSG00000227029
ENSG00000239708,ENSG00000239708,RN7SL782P
ENSG00000274532,ENSG00000274532,ENSG00000274532


In [38]:
adata.var.index = adata.var.index.astype('str')
adata.var.dtypes

ensembl_id     object
gene_symbol    object
dtype: object

# Check raw data

In [ ]:
adata.X.toarray().max()

22619.0

In [39]:
# make checks for adata before saving raw
adata.obs = adata.obs.astype(str)

# Make specific columns numeric
adata.obs['cell_viability_percentage'] = pd.to_numeric(adata.obs['cell_viability_percentage'], errors='coerce')
adata.obs['cell_number_loaded'] = pd.to_numeric(adata.obs['cell_number_loaded'], errors='coerce')
adata.obs['sample_collection_year'] = pd.to_numeric(adata.obs['sample_collection_year'], errors='coerce')


numeric_cols = ['nGene', 'nUMI', 'percent.mito']
for col in numeric_cols:
    adata.obs[col] = pd.to_numeric(adata.obs[col], errors='coerce')

# Make df indices str
adata.obs.index = adata.obs.index.astype('str')
adata.var.index = adata.var.index.astype('str')
adata.var_names = adata.var_names.astype('str')

In [40]:
adata.X = adata.X.astype(np.int64)

In [41]:
adata.raw = adata

In [ ]:
adata.X

<17867x46204 sparse matrix of type '<class 'numpy.int64'>'
	with 39438833 stored elements in Compressed Sparse Row format>

In [ ]:
adata.raw.X

<17867x46204 sparse matrix of type '<class 'numpy.int64'>'
	with 39438833 stored elements in Compressed Sparse Row format>

In [ ]:
adata.X.toarray().max()

22619

In [ ]:
adata.raw.X.toarray().max()

22619

## Validation result

### UNS Validation
- OK: Tier 1 UNS Google Sheet
- OK: Tier 1 UNS AnnData Object  same as Google Sheet

### OBS Validation
- OK: Tier 1 OBS Google Sheet
- OK: Tier 1 OBS AnnData Object same as Google Sheet


## Not ideal:

sample_source, sample_collection_method, sequenced_fragment, and intron_inclusion

contain NAs

# Data Submission Status

- CHECK: Raw counts in X and in raw
- CHECK: Tier 1 Metadata in OBS
- CHECK: Cell Ontology IDs in OBS
- CHECK: Author cell type in OBS
- CHECK: Marker genes in UNS
- CHECK: ENSEMBL IDs and gene symbols in var

# Revisions:

- Find out NAs about sample_source, sample_collection_method, sequenced_fragment, and intron_inclusion

- ENSEMBL IDs have been queryied -> Maybe ask at some point again

In [42]:
adata.write_h5ad(join(OUTPUT_PATH_PREREVISION, f"{DATASET_ID}.h5ad"))
adata.write_h5ad(join(OUTPUT_PATH_POSTREVISION, f"{DATASET_ID}.h5ad"))